In [1]:
import pandas as pd
import jieba

In [2]:
df = pd.read_excel("data.xlsx")
words = df["句式"].tolist()
tags = df["知识点"].tolist()
words[:5],tags[:5]

(['喂，你好，是林家敬先生吗？喂林先生听得到吗？',
  '潘晓峰是吧？',
  '喂，你好，是张红园是吧？喂。喂，你好是张红园张先生是吗？是不是啊先生您说个话呀。您不说话也解决不了问题好吗？',
  '喂喂，小敏先生吗？是小明先生吗？',
  '噢，我说要处理的喂你好。'],
 ['表明身份', '表明身份', '表明身份', '表明身份', '表明身份'])

In [3]:
f = open("test_data.txt","r",encoding="utf-8")
lines_lst = f.readlines()
test_words = [ item.split("\t")[0] for item in lines_lst]
test_tag = [item.split("\t")[1].replace("\n","") for item in lines_lst ]
test_words[:5],test_tag[:5]

(['您是在上学还是在上班？', '喂，你好，是刘忠先生吗？', '闪银至尊的先生。', '那上一个月先生这个月不一样了。', '喂你好。'],
 ['拒识', '表明身份', '表明身份', '逾期', '拒识'])

###  停用词 标点处理 ###

In [5]:
words = [ item.replace("。","").replace("，","").replace("？","").replace("！","") for item in words]
words[:5],tags[:5]
test_words =[ item.replace("。","").replace("，","").replace("？","").replace("！","") for item in test_words]
test_words[:5],test_tag[:5]

(['您是在上学还是在上班', '喂你好是刘忠先生吗', '闪银至尊的先生', '那上一个月先生这个月不一样了', '喂你好'],
 ['拒识', '表明身份', '表明身份', '逾期', '拒识'])

In [6]:
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [7]:
words = [ " ".join(jieba.lcut(item)) for item in words]
test_words = [" ".join(jieba.lcut(item)) for item in test_words]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 1.495 seconds.
Prefix dict has been built successfully.


In [9]:
words[:5],test_words[:5]

(['喂 你好 是 林家 敬 先生 吗 喂 林先生 听 得到 吗',
  '潘晓峰 是 吧',
  '喂 你好 是 张 红园 是 吧 喂喂 你好 是 张 红园 张 先生 是 吗 是不是 啊 先生 您 说 个 话 呀 您 不 说话 也 解决不了 问题 好 吗',
  '喂喂 小敏 先生 吗 是 小明 先生 吗',
  '噢 我 说 要 处理 的 喂 你好'],
 ['您 是 在 上学 还是 在 上班',
  '喂 你好 是 刘忠 先生 吗',
  '闪银 至尊 的 先生',
  '那上 一个月 先生 这个 月 不 一样 了',
  '喂 你好'])

In [10]:
label = list(set(tags))
label = { item:index for index,item in enumerate(label)}
label

{'加微信': 0, '拒识': 1, '表述欠款金额': 2, '逾期': 3, '表明身份': 4, '微信公众号': 5}

In [11]:
train_data = words[:]
train_labels = [ label[item] for item in tags]
assert(len(train_data)==len(train_labels))
print("total_nums...",len(train_data))

total_nums... 1808


In [12]:
test_data = test_words[:]
test_labels = [ label[item] for item in test_tag ]
assert(len(test_data)==len(test_labels))
print("test_nums...",len(test_labels))

test_nums... 349


In [14]:
x_train, x_test, y_train, y_test = train_data,test_data,train_labels,test_labels

vectorizer = CountVectorizer(max_features=5000)
tf_idf_transformer = TfidfTransformer()
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(x_train))
x_train_weight = tf_idf.toarray()  # 训练集TF-IDF权重矩阵
tf_idf = tf_idf_transformer.transform(vectorizer.transform(x_test))
x_test_weight = tf_idf.toarray()  # 测试集TF-IDF权重矩阵

In [15]:
# 将数据转化为DMatrix类型
dtrain = xgb.DMatrix(x_train_weight, label=y_train)
dtest = xgb.DMatrix(x_test_weight, label=y_test)


In [18]:
param = {'silent': 0, 'eta': 0.2, 'max_depth': 6, 'objective': 'multi:softmax', 'num_class': 6, 'eval_metric': 'mlogloss'}  # 参数
evallist = [(dtrain, 'train'), (dtest, 'test')]
num_round = 500  # 循环次数
xgb_model = xgb.train(param, dtrain, num_round,evallist)

[17:40:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:1.55296	test-mlogloss:1.55266
[1]	train-mlogloss:1.39409	test-mlogloss:1.39729
[2]	train-mlogloss:1.26918	test-mlogloss:1.27515
[3]	train-mlogloss:1.17516	test-mlogloss:1.18415
[4]	train-mlogloss:1.09795	test-mlogloss:1.10868
[5]	train-mlogloss:1.03509	test-mlogloss:1.04581
[6]	train-mlogloss:0.98086	test-mlogloss:0.99620
[7]	train-mlogloss:0.93259	test-mlogloss:0.94936
[8]	train-mlogloss:0.89206	test-mlogloss:0.90914
[9]	train-mlogloss:0.85670	test-mlogloss:0.87835
[10]	train-mlogloss:0.82644	test-mlogloss:0.84605
[11]	train-mlogloss:0.80066	test-mlogloss:0.82145
[12]	train-mlogloss:0.7762

[156]	train-mlogloss:0.32033	test-mlogloss:0.35349
[157]	train-mlogloss:0.31935	test-mlogloss:0.35271
[158]	train-mlogloss:0.31859	test-mlogloss:0.35203
[159]	train-mlogloss:0.31780	test-mlogloss:0.35134
[160]	train-mlogloss:0.31706	test-mlogloss:0.35036
[161]	train-mlogloss:0.31624	test-mlogloss:0.34959
[162]	train-mlogloss:0.31559	test-mlogloss:0.34889
[163]	train-mlogloss:0.31446	test-mlogloss:0.34795
[164]	train-mlogloss:0.31357	test-mlogloss:0.34723
[165]	train-mlogloss:0.31260	test-mlogloss:0.34601
[166]	train-mlogloss:0.31196	test-mlogloss:0.34555
[167]	train-mlogloss:0.31127	test-mlogloss:0.34499
[168]	train-mlogloss:0.31019	test-mlogloss:0.34386
[169]	train-mlogloss:0.30935	test-mlogloss:0.34308
[170]	train-mlogloss:0.30863	test-mlogloss:0.34181
[171]	train-mlogloss:0.30778	test-mlogloss:0.34069
[172]	train-mlogloss:0.30692	test-mlogloss:0.33975
[173]	train-mlogloss:0.30634	test-mlogloss:0.33911
[174]	train-mlogloss:0.30566	test-mlogloss:0.33838
[175]	train-mlogloss:0.30482	te

[317]	train-mlogloss:0.23414	test-mlogloss:0.25834
[318]	train-mlogloss:0.23389	test-mlogloss:0.25810
[319]	train-mlogloss:0.23362	test-mlogloss:0.25788
[320]	train-mlogloss:0.23335	test-mlogloss:0.25754
[321]	train-mlogloss:0.23298	test-mlogloss:0.25726
[322]	train-mlogloss:0.23269	test-mlogloss:0.25687
[323]	train-mlogloss:0.23230	test-mlogloss:0.25645
[324]	train-mlogloss:0.23197	test-mlogloss:0.25604
[325]	train-mlogloss:0.23169	test-mlogloss:0.25592
[326]	train-mlogloss:0.23144	test-mlogloss:0.25559
[327]	train-mlogloss:0.23120	test-mlogloss:0.25536
[328]	train-mlogloss:0.23100	test-mlogloss:0.25505
[329]	train-mlogloss:0.23074	test-mlogloss:0.25504
[330]	train-mlogloss:0.23049	test-mlogloss:0.25496
[331]	train-mlogloss:0.23027	test-mlogloss:0.25451
[332]	train-mlogloss:0.23005	test-mlogloss:0.25435
[333]	train-mlogloss:0.22967	test-mlogloss:0.25381
[334]	train-mlogloss:0.22937	test-mlogloss:0.25333
[335]	train-mlogloss:0.22912	test-mlogloss:0.25293
[336]	train-mlogloss:0.22887	te

[478]	train-mlogloss:0.20237	test-mlogloss:0.22397
[479]	train-mlogloss:0.20223	test-mlogloss:0.22372
[480]	train-mlogloss:0.20210	test-mlogloss:0.22358
[481]	train-mlogloss:0.20198	test-mlogloss:0.22359
[482]	train-mlogloss:0.20189	test-mlogloss:0.22341
[483]	train-mlogloss:0.20181	test-mlogloss:0.22333
[484]	train-mlogloss:0.20169	test-mlogloss:0.22319
[485]	train-mlogloss:0.20155	test-mlogloss:0.22309
[486]	train-mlogloss:0.20145	test-mlogloss:0.22295
[487]	train-mlogloss:0.20133	test-mlogloss:0.22268
[488]	train-mlogloss:0.20125	test-mlogloss:0.22245
[489]	train-mlogloss:0.20114	test-mlogloss:0.22235
[490]	train-mlogloss:0.20101	test-mlogloss:0.22222
[491]	train-mlogloss:0.20092	test-mlogloss:0.22221
[492]	train-mlogloss:0.20076	test-mlogloss:0.22195
[493]	train-mlogloss:0.20061	test-mlogloss:0.22183
[494]	train-mlogloss:0.20050	test-mlogloss:0.22162
[495]	train-mlogloss:0.20037	test-mlogloss:0.22150
[496]	train-mlogloss:0.20026	test-mlogloss:0.22143
[497]	train-mlogloss:0.20016	te

In [19]:
y_predict = xgb_model.predict(dtest)  # 模型预测
label_all = ['逾期', '表述欠款金额','微信公众号',"表明身份","加微信","拒识"]
confusion_mat = metrics.confusion_matrix(y_test, y_predict)
df = pd.DataFrame(confusion_mat, columns=label_all)
df.index = label_all
print('准确率：', metrics.accuracy_score(y_test, y_predict))
print('confusion_matrix:', df)
print('分类报告:', metrics.classification_report(y_test, y_predict))

准确率： 0.9426934097421203
confusion_matrix:         逾期  表述欠款金额  微信公众号  表明身份  加微信  拒识
逾期       6       0      0     3    0   0
表述欠款金额   0      70      0     0    0   0
微信公众号    0       1     48     0    0   0
表明身份     0       8      0   158    0   0
加微信      0       7      0     1   41   0
拒识       0       0      0     0    0   6
分类报告:               precision    recall  f1-score   support

           0       1.00      0.67      0.80         9
           1       0.81      1.00      0.90        70
           2       1.00      0.98      0.99        49
           3       0.98      0.95      0.96       166
           4       1.00      0.84      0.91        49
           5       1.00      1.00      1.00         6

    accuracy                           0.94       349
   macro avg       0.96      0.91      0.93       349
weighted avg       0.95      0.94      0.94       349

